<a href="https://colab.research.google.com/github/HWANG-HOE-SUN/Univ.Project/blob/main/%20MCR_%EB%B6%84%EC%84%9D/MCR%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/') 

Mounted at /content/gdrive/


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
import time

In [6]:
folder = '/content/gdrive/MyDrive/MCR데이터'

In [7]:
file_list = os.listdir(folder)
file_list

['2019년 소비자행태조사 원시데이터_문항,보기.xlsx',
 '2019년 소비자행태조사 원시데이터_답변.csv']

In [8]:
file_name = []
for file in file_list:
  name = folder+'/'+file
  file_name.append(name)

In [47]:
# 원본 항목, 답변
mcr_raw_quest = pd.read_excel(file_name[0])
mcr_raw_answer = pd.read_csv(file_name[1])
# 항목 설명 정보
mcr_raw_info = pd.read_excel(file_name[0],sheet_name='보기정보')

mcr_raw_quest.head(3), print('\n'), mcr_raw_answer.head(3), print('\n'), mcr_raw_info.head(3)

(    항목ID      항목명  항목분류 항목유형            항목설명
 0   YEAR     조사년도  DEMO   단수            <없음>
 1  A_001       성별  DEMO   단수  귀하의 성별은 무엇입니까?
 2  A_002  연령(10대)  DEMO   단수  귀하의 연령은 무엇입니까?,
 None,
    YEAR  A_001  A_002  A_003  ...  K_05_015#7  K_05_015#8  K_05_015#9  K_05_015#10
 0  2019      1      1     14  ...         NaN         NaN         NaN          NaN
 1  2019      1      1     14  ...         NaN         NaN         NaN          NaN
 2  2019      1      1     14  ...         NaN         NaN         NaN          NaN
 
 [3 rows x 3624 columns],
 None,
    항목ID  항목코드  항목코드명
 0  YEAR  2001  2001년
 1  YEAR  2002  2002년
 2  YEAR  2003  2003년)

In [10]:
mcr_raw_answer.describe()

,YEAR,A_001,A_002,A_003,A_004,A_006,A_007,A_009,A_011,A_013,A_014,A_016,A_017,A_018,A_019,A_026,A_027,A_034,B_01_001,B_01_003,B_01_005,B_01_013,B_01_015,B_01_017,B_01_019,B_01_021,B_01_025,B_01_026,B_01_029,B_03_001,B_03_002,B_03_003,B_03_004,B_03_005,B_03_006,B_03_007,B_03_008,B_03_009,B_03_020,B_03_021,...,K_05_005#1,K_05_005#2,K_05_006#1,K_05_006#2,K_05_007#1,K_05_007#2,K_05_007#3,K_05_007#4,K_05_007#5,K_05_007#6,K_05_007#7,K_05_007#8,K_05_007#9,K_05_007#10,K_05_008#1,K_05_008#2,K_05_009#1,K_05_009#2,K_05_010#1,K_05_010#2,K_05_011#1,K_05_011#2,K_05_012#1,K_05_012#2,K_05_013#1,K_05_013#2,K_05_014#1,K_05_014#2,K_05_014#3,K_05_014#4,K_05_015#1,K_05_015#2,K_05_015#3,K_05_015#4,K_05_015#5,K_05_015#6,K_05_015#7,K_05_015#8,K_05_015#9,K_05_015#10
count,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,2417.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,2476.000000,2486.000000,4000.000000,4000.000000,4000.000000,4000.000000,1151.000000,3866.000000,3866.000000,4000.00000,4000.000000,4000.000000,627.000000,3839.000000,3839.000000,3839.000000,3785.000000,3785.000000,3785.000000,3599.000000,3599.000000,3599.000000,1151.000000,1151.000000,...,4000.000000,3961.000000,4000.00000,3956.000000,4000.000000,3972.000000,3818.000000,3496.000000,2855.000000,2176.000000,1485.000000,842.000000,528.000000,314.000000,4000.000000,3973.000000,4000.000000,3983.000000,4000.000000,3985.000000,4000.000000,3983.000000,4000.000000,3984.000000,4000.000000,3980.000000,4000.000000,2290.000000,832.000000,271.000000,4000.00000,3235.000000,1767.000000,796.000000,294.000000,106.00000,31.000000,9.000000,5.0,0.0
mean,2019.0,1.494500,3.666500,41.394500,2.945000,3.979000,5.645500,3.626810,3.318250,5.72725,1.395750,1.268750,2.143750,1.504500,1.412000,1.619000,1.841276,1.592920,1.048500,1.065500,1.138000,1.913500,1.084275,1.367822,1.016037,1.84325,1.645250,1.855250,2.236045,94.186767,123.430841,131.569940,65.246764,83.847556,86.039630,59.339261,76.028619,77.905807,64.502172,39.178975,...,4.501750,6.155769,4.48450,6.435035,1.695500,2.832578,4.366684,5.856693,6.799299,7.505515,8.078788,8.475059,9.071970,9.917197,2.497750,5.068966,3.770500,5.950540,4.271000,6.515182,4.271500,6.607080,4.423250,6.861446,2.177750,2.889196,2.341000,2.346725,3.215144,3.996310,2.23175,4.144359,5.595359,6.530151,7.312925,7.90566,8.290323,8.444444,9.0,NaN
std,0.0,0.500032,1.540413,15.151741,1.758621,1.710933,2.870608,1.442522,12.611948,2.20365,0.489072,0.567982,0.743454,0.868141,0.556177,0.485693,0.549976,0.676134,0.214847,0.247437,0.344943,0.281136,0.277920,0.482275,0.125635,0.36361,0.478497,0.351893,0.729420,62.743685,75.845197,82.364486,42.375029,56.018682,61.380713,42.643319,50.209389,53.402004,76.988332,60.380512,...,6.666004,2.548266,6.53138,2.557076,6.434526,1.551895,1.893458,1.877904,1.743744,1.579747,1.371097,1.134255,0.883396,0.529775,6.405798,2.677397,6.064406,2.424203,5.978294,2.568701,6.008645,2.628996,6.029983,2.532417,5.226565,2.020387,8.967766,0.602420,0.419858,0.060746,6.27765,1.883068,1.785445,1.581448,1.315711,0.94141,0.739078,0.527046,0.0,NaN
min,2019.0,1.000000,1.000000,13.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,3.000000,4.000000,6.00000,7.000000,8.000000,9.0,NaN
25%,2019.0,1.000000,2.000000,28.000000,2.000000,3.000000,4.000000,3.000000,1.000000,4.00000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.00

In [11]:
mcr_raw_quest.groupby(['항목분류']).count()

,항목ID,항목명,항목유형,항목설명
항목분류,,,,
DEMO,18,18,18,18
기타>나쁜광고,56,56,56,56
기타>소비자경제동향,15,15,15,15
라이프스타일>심리적 태도,90,90,90,90
라이프스타일>여가활동,96,96,96,96
...,...,...,...,...
제품>패스트푸드/프랜차이즈>햄버거/샌드위치 전문점,31,31,31,31
제품>화장품/향수/여성용품>남성기초화장품,31,31,31,31
제품>화장품/향수/여성용품>미용종합,12,12,12,12


### raw_quest(질문 항목) 제품별 분류

In [48]:
import copy
mcr_raw_quest_copy = copy.deepcopy(mcr_raw_quest) # 별도의 새로운 객체 생성

In [49]:
mcr_raw_quest_copy['제품항목여부'] = mcr_raw_quest_copy.항목분류.apply(lambda x: 1 if '제품' in x else 0)

In [50]:
mcr_raw_quest_copy['제품항목여부'] = mcr_raw_quest_copy.항목설명.apply(lambda x: 1 if '제품' in x else 0)

In [51]:
jepum_quests = mcr_raw_quest_copy[mcr_raw_quest_copy['제품항목여부']==1]
jepum_quests

,항목ID,항목명,항목분류,항목유형,항목설명,제품항목여부
350,C_01_019,PPL(간접광고) 평가 - 제품/브랜드 관심도,매체이용행태>TV종합,단수,귀하께서는 PPL광고(간접광고)에 대해 다음의 항목별로 어떻게 생각하십니까? - 제...,1
351,C_01_020,PPL(간접광고) 평가 - 제품/브랜드 인지,매체이용행태>TV종합,단수,귀하께서는 PPL광고(간접광고)에 대해 다음의 항목별로 어떻게 생각하십니까? - 제...,1
352,C_01_021,PPL(간접광고) 평가 - 제품/브랜드의 이미지 향상,매체이용행태>TV종합,단수,귀하께서는 PPL광고(간접광고)에 대해 다음의 항목별로 어떻게 생각하십니까? - 제...,1
353,C_01_022,PPL(간접광고) 평가 - 제품/브랜드 추가 검색 의향,매체이용행태>TV종합,단수,귀하께서는 PPL광고(간접광고)에 대해 다음의 항목별로 어떻게 생각하십니까? - 제...,1
354,C_01_023,PPL(간접광고) 평가 - 제품/브랜드에 대한 정보 공유 의향,매체이용행태>TV종합,단수,귀하께서는 PPL광고(간접광고)에 대해 다음의 항목별로 어떻게 생각하십니까? - 제...,1
...,...,...,...,...,...,...
3539,J_31_005,사물인터넷 기능 추가 비용 지불 의향,제품>가전종합,단수,"귀하의 댁에 설치된 여러 가지 가전제품에 인터넷을 연결 하여, 집 밖에서도 조작을 ...",1
3540,J_31_006,신기술 제품 구매 시점,제품>가전종합,단수,"귀하는 예전에 없던 기술과 기능을 갖춘 가전제품이 새로 등장한다면, 어떻게 구매하실...",1
3541,J_31_007,프리미엄 가전 구매 제품,제품>가전종합,단수,귀하는 일반 제품보다 고가에 판매되는 프리미엄 가전 제 품을 구매할 생각이 있습니까?,1
3546,J_32_004,B급 식품 할인판매 구매 의향,제품>식품종합,단수,"귀하는 식품의 신선도에는 문제가 없지만, 외관상 상처가 있거나 반품되어 할인판매하는...",1


In [52]:
jepum_quests_id = jepum_quests.항목ID
jepum_quests_id

350     C_01_019
351     C_01_020
352     C_01_021
353     C_01_022
354     C_01_023
          ...   
3539    J_31_005
3540    J_31_006
3541    J_31_007
3546    J_32_004
3549    J_32_007
Name: 항목ID, Length: 331, dtype: object

In [53]:
jepum_answers = mcr_raw_answer[list(jepum_quests_id)]
jepum_answers

,C_01_019,C_01_020,C_01_021,C_01_022,C_01_023,C_01_024,C_01_031#1,C_01_031#2,C_01_031#3,C_01_031#4,C_01_031#5,C_01_031#6,C_01_031#7,C_01_031#8,C_01_031#9,C_01_031#10,C_01_031#11,C_01_031#12,C_01_031#13,C_01_031#14,C_01_031#15,C_01_031#16,C_01_031#17,C_01_031#18,C_01_031#19,C_01_031#20,C_01_031#21,C_01_032,C_01_033,C_01_034#1,C_01_034#2,C_01_034#3,C_01_034#4,C_01_034#5,C_01_034#6,C_01_034#7,C_01_034#8,C_01_034#9,C_01_034#10,C_01_034#11,...,I_099,I_100,I_101,I_102,I_103,I_104,I_105,I_106,I_107,I_108,I_109,I_110,I_111,I_112,I_113,I_114,I_115,I_116,I_117,J_03_02_003#1,J_03_02_003#2,J_03_04_003#1,J_03_04_003#2,J_15_01_009,J_15_01_010,J_15_01_011,J_15_01_012,J_15_01_015,J_15_01_016,J_25_19_008,J_30_05_002,J_31_001,J_31_002,J_31_003,J_31_004,J_31_005,J_31_006,J_31_007,J_32_004,J_32_007
0,6,6,6,4,5,4,1,2.0,3.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,5,3,3,6,4,4,5,3,6,4,4,3,2,5,3,4,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,2,4,2,2,2,2,2,2
1,3,4,2,2,2,2,5,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,5,4,4,4,4,4,4,4,2,2,2,2,2,2,2,4,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,3,3,4,3,2,2,2,1,3
2,5,4,5,6,5,5,1,2.0,3.0,5.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,4,4,5,6,5,5,4,5,5,6,5,5,4,5,5,6,5,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,3,1,3,2,1,2,2,3
3,2,2,2,2,2,2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,3,3,3,1,2,2,2,3
4,6,6,5,4,4,3,3,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,5,4,4,6,3,5,4,4,6,3,5,4,4,6,3,5,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,3,4,3,2,2,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,4,5,4,3,4,4,1,2.0,4.0,5.0,7.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,5.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,4,4,5,5,4,3,4,3,3,3,4,4,3,5,4,4,5,5,2.0,NaN,2.0,NaN,10.0,90.0,2.0,1.0,2.0,2.0,1.0,3,3,4,1,4,2,2,1,2,3
3996,3,3,3,4,3,4,2,8.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,3,2,3,2,3,2,3,2,3,4,3,4,4,3,3,2,3,3,2.0,NaN,2.0,NaN,0.0,100.0,2.0,2.0,1.0,2.0,3.0,3,3,3,1,5,2,2,2,1,3
3997,4,3,5,4,3,4,1,2.0,6.0,7.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,3,4,5,4,4,5,5,3,3,4,5,4,4,3,4,4,3,4,1.0,NaN,1.0,NaN,20.0,80.0,2.0,1.0,2.0,2.0,3.0,3,1,4,1,4,2,2,2,2,3
3998,3,4,3,4,3,4,2,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,3,2,3,3,2,3,2,3,2,3,2,3,2,3,3,3,3,3,2.0,NaN,2.0,NaN,NaN,NaN,2.0,1.0,1.0,2.0,3.0,3,1,3,1,3,2,2,2,2,3


In [ ]:
# jepum_cat = [
# '시사, 사회 뉴스',
# '부동산, 증권, 금융',
# '연예, 오락, 영화',
# '요리, 육아, 생활정보',
# '건강, 의학',
# '퀴즈, 게임',
# '스포츠',
# '쇼핑',
# '자동차',
# '기타'           
# ]
# jepum_cat

In [55]:
mcr_raw_quest_copy[mcr_raw_quest_copy.항목분류=='제품 광고 관심도'].항목ID.head(3)

1249    I_001
1250    I_002
1251    I_012
Name: 항목ID, dtype: object

### 제품별 광고 관심도 추출

In [56]:
jepum_adver_cat = mcr_raw_quest_copy[mcr_raw_quest_copy.항목분류=='제품 광고 관심도'].항목설명.apply(lambda x: str(x).split('-')[-1])
jepum_adver_cat

1249      휴대폰 단말기 광고
1250     이동통신 서비스 광고
1251         골프웨어 광고
1252       아웃도어의류 광고
1253       스포츠캐주얼 광고
            ...     
1310       뷰티디바이스 광고
1311        섬유유연제 광고
1312           우유 광고
1313           카페 광고
1314        햄버거/샌드위치
Name: 항목설명, Length: 66, dtype: object

In [57]:
jepum_adver_df = mcr_raw_answer[mcr_raw_quest_copy[mcr_raw_quest_copy.항목분류=='제품 광고 관심도'].항목ID]
jepum_adver_df.columns = jepum_adver_cat
jepum_adver_df.head(3)

항목설명,휴대폰 단말기 광고,이동통신 서비스 광고,골프웨어 광고,아웃도어의류 광고,스포츠캐주얼 광고,유아 아동용 화장품/바디용품 광고,신용카드 광고,모바일 게임 광고,구인구직 서비스 광고,자가용 승용차 광고,정부기관 및 공기업 광고,기업이미지 광고,항공사 광고,은행 광고,"리스, 렌탈, 카쉐어링 광고",에어컨 광고,세탁기 광고,TV 광고,냉장고 광고,청소기 광고,의류관리(스타일러) 광고,건조기 광고,정수기 광고,공기청정기 광고,침대 광고,침구 광고,보일러 광고,온수매트 광고,잇몸 관리용품 광고,관절염 치료제 광고,감기약 광고,상처/흉터치료제 광고,두통약 광고,위장약/소화제/변비약 광고,맥주 광고,샴푸/린스 광고,탈모 관련 광고,세탁세제 광고,부동산 중개 서비스 광고,여행사/여행 숙박 중개사 광고,해외여행 광고,웹콘텐츠 광고,기능성 음료 광고,탄산음료 광고,생수 광고,탄산수 광고,요구르트 광고,캔커피 광고,인스턴트 커피 광고,음식배달 식재료 광고,피자전문점 광고,치킨전문점 광고,라면 광고,간편식 광고,교육 광고,종합영양제 광고,건강보조식품 광고,남성용 화장품 광고,여성용 화장품 광고,간편결제 서비스 광고,O2O 서비스 광고,뷰티디바이스 광고,섬유유연제 광고,우유 광고,카페 광고,햄버거/샌드위치
0,2,5,3,6,4,4,5,3,5,4,4,6,3,5,4,4,2,5,3,4,3,3,5,2,6,3,4,4,1,5,2,6,3,5,4,4,2,5,3,4,4,3,5,5,5,3,6,4,5,3,3,6,4,4,5,3,6,4,4,3,2,5,3,4,4,1
1,2,2,2,1,2,2,2,2,2,3,3,3,3,3,2,2,2,1,1,1,3,2,1,1,2,2,2,1,1,2,1,5,5,5,5,4,4,4,4,4,4,4,4,4,5,3,4,5,5,4,4,4,4,4,4,4,2,2,2,2,2,2,2,4,3,3
2,5,4,5,6,5,5,4,5,6,5,4,5,3,4,5,4,4,5,6,5,4,3,4,4,5,4,5,6,5,4,3,4,5,4,6,5,4,3,4,5,4,5,4,5,4,5,6,5,4,4,5,6,5,5,4,5,5,6,5,5,4,5,5,6,5,4


In [58]:
jepum_adver_df.isnull().sum() # null값은 없음

항목설명
 휴대폰 단말기 광고     0
 이동통신 서비스 광고    0
 골프웨어 광고        0
 아웃도어의류 광고      0
 스포츠캐주얼 광고      0
               ..
 뷰티디바이스 광고      0
 섬유유연제 광고       0
 우유 광고          0
 카페 광고          0
 햄버거/샌드위치       0
Length: 66, dtype: int64

In [59]:
jepum_adver_df.describe()

항목설명,휴대폰 단말기 광고,이동통신 서비스 광고,골프웨어 광고,아웃도어의류 광고,스포츠캐주얼 광고,유아 아동용 화장품/바디용품 광고,신용카드 광고,모바일 게임 광고,구인구직 서비스 광고,자가용 승용차 광고,정부기관 및 공기업 광고,기업이미지 광고,항공사 광고,은행 광고,"리스, 렌탈, 카쉐어링 광고",에어컨 광고,세탁기 광고,TV 광고,냉장고 광고,청소기 광고,의류관리(스타일러) 광고,건조기 광고,정수기 광고,공기청정기 광고,침대 광고,침구 광고,보일러 광고,온수매트 광고,잇몸 관리용품 광고,관절염 치료제 광고,감기약 광고,상처/흉터치료제 광고,두통약 광고,위장약/소화제/변비약 광고,맥주 광고,샴푸/린스 광고,탈모 관련 광고,세탁세제 광고,부동산 중개 서비스 광고,여행사/여행 숙박 중개사 광고,해외여행 광고,웹콘텐츠 광고,기능성 음료 광고,탄산음료 광고,생수 광고,탄산수 광고,요구르트 광고,캔커피 광고,인스턴트 커피 광고,음식배달 식재료 광고,피자전문점 광고,치킨전문점 광고,라면 광고,간편식 광고,교육 광고,종합영양제 광고,건강보조식품 광고,남성용 화장품 광고,여성용 화장품 광고,간편결제 서비스 광고,O2O 서비스 광고,뷰티디바이스 광고,섬유유연제 광고,우유 광고,카페 광고,햄버거/샌드위치
count,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000
mean,3.30325,3.289500,3.685250,2.996000,3.393500,3.330500,3.466500,3.40850,3.364750,3.334000,3.271500,3.27975,3.420250,3.286000,3.130000,2.952000,2.927250,3.000000,2.989000,3.101250,3.186000,3.207500,3.229500,3.532750,3.315250,3.110000,3.074750,3.078000,2.994500,2.960000,3.437250,3.449500,3.659000,3.682750,3.519750,2.973750,2.881000,3.088500,3.160250,3.252500,3.057250,3.296000,3.247500,3.38550,3.485500,3.490750,3.504750,3.552500,3.688000,3.634250,3.465000,3.389750,3.104500,3.374500,3.517500,3.083500,3.389500,3.386250,3.016250,3.168500,3.284500,2.79300,3.071750,3.083000,3.047000,2.833500
std,1.03974,1.083736,1.157159,1.181877,1.086486,1.125205,1.122137,1.16188,1.161914,1.210913,1.199645,1.12085,1.185112,1.147839,1.141458,1.176453,1.169316,1.216293,1.214186,1.141851,1.142904,1.131708,1.135408,1.207391,1.153777,1.205306,1.181318,1.213377,1.186306,1.158549,1.166577,1.150122,1.145664,1.138388,1.133548,1.279441,1.233783,1.101804,1.110125,1.096834,1.108954,1.064973,1.072162,1.04075,1.062578,1.122822,1.106021,1.122969,1.104745,1.085482,1.118073,1.105509,1.150396,1.186635,1.150445,1.168918,1.148094,1.173203,1.229375,1.130454,1.271599,1.16447,1.075821,1.091975,1.092286,1.057619
min,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000
25%,3.00000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.00000,3.000000,2.000000,2.000000,3.00000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000
50%,3.00000,3.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.0000

### '영향력'을 묻는 문항 추출

In [60]:
temp = mcr_raw_quest_copy.항목설명.apply(lambda x: True if '영향력' in str(x) else False)
temp = temp[temp==1].index
mcr_impact_quest = mcr_raw_quest.iloc[temp]
mcr_impact_quest_id = mcr_impact_quest.항목ID
mcr_impact_quest

,항목ID,항목명,항목분류,항목유형,항목설명
1371,J_01_02_014,주거래 은행 선택 시 광고 영향력,제품>금융>은행,단수,"주거래 은행 선택 시, 광고의 영향력은 어느 정도입니까?"
1403,J_01_04_014,신용카드 선택 시 광고의 영향력,제품>금융>신용카드,단수,"신용카드 선택 시, 광고의 영향력은 어느 정도입니까?"
1438,J_01_05_009,모바일 간편결제 서비스 선택 시 광고의 영향력,제품>금융>간편결제,단수,"모바일 간편결제 브랜드 선택 시, 광고의 영향력은 어느 정도입니까?"
1473,J_02_01_008,휴대폰 선택시 광고의 영향력,제품>통신>휴대폰,단수,"휴대폰 선택 시, 광고의 영향력은 어느 정도입니까?"
1526,J_02_04_012,이동통신사 선택시 광고의 영향력,제품>통신>이동통신 서비스,단수,"이동통신사 선택 시, 광고의 영향력은 어느 정도입니까?"
...,...,...,...,...,...
3352,J_27_01_011,모바일게임 선택시 광고의 영향력,제품>온라인 콘텐츠>모바일 게임,단수,"모바일 게임 선택 시, 광고의 영향력은 어느 정도입니까?"
3383,J_27_03_007,웹콘텐츠 플랫폼 선택시 광고의 영향력,제품>온라인 콘텐츠>웹콘텐츠,단수,"웹툰 플랫폼 선택 시, 광고의 영향력은 어느 정도입니까?"
3427,J_28_01_022,O2O 서비스 앱 선택시 광고 영향력,제품>O2O 서비스>o2o,단수,서비스 앱을 선택하실 때 광고가 미치는 영향력은 어느 정도입니까?
3472,J_28_03_007,구인구직 서비스 브랜드 선택 시 광고의 영향력,제품>O2O 서비스>구인구직 서비스,단수,"구인구직 서비스 브랜드 선택 시, 광고의 영향력은 어느 정도입니까?"


In [61]:
mcr_impact_answer = mcr_raw_answer.loc[:,mcr_impact_quest_id]
mcr_impact_answer.describe() # null값이 다수 존재함

,J_01_02_014,J_01_04_014,J_01_05_009,J_02_01_008,J_02_04_012,J_02_04_035,J_03_01_004,J_03_02_005,J_03_04_005,J_03_05_003,J_03_07_003,J_03_09_004,J_03_14_004,J_03_15_003,J_03_16_004,J_05_01_007,J_05_05_007,J_05_06_010,J_05_18_011,J_06_01_013,J_06_02_014,J_09_02_008,J_09_03_007,J_09_05_007,J_10_02_008,J_10_03_008,J_10_08_007,J_12_01_008,J_12_04_009,J_13_01_010,J_13_02_007,J_14_02_010,J_14_06_006,J_16_02_011,J_16_03_010,J_17_01_009,J_17_02_009,J_17_03_006,J_17_06_006,J_17_08_009,J_17_10_006,J_18_02_015,J_19_01_004,J_19_06_006,J_20_03_003,J_20_04_004,J_20_05_003,J_21_02_004,J_22_01_008,J_22_03_010,J_22_03_014,J_23_08_007,J_23_11_008,J_23_12_008,J_24_05_006,J_25_02_013,J_25_10_013,J_25_20_004,J_26_01_005,J_27_01_011,J_27_03_007,J_28_01_022,J_28_03_007,J_28_04_006
count,3620.000000,3463.000000,1507.000000,3994.000000,3994.000000,323.000000,3629.000000,3634.000000,3658.000000,3531.000000,3494.000000,2191.000000,138.000000,444.000000,1451.000000,2921.000000,614.000000,2003.000000,2684.000000,2575.000000,1641.000000,2950.000000,1813.000000,2955.000000,2077.000000,3323.000000,2108.000000,3665.000000,1636.000000,2124.000000,1934.000000,3491.000000,446.000000,1279.000000,839.000000,679.000000,986.000000,695.000000,511.000000,581.000000,682.000000,2606.000000,3014.000000,216.000000,3076.000000,900.000000,804.000000,3392.000000,1079.000000,1132.000000,1421.000000,398.000000,1254.000000,2395.000000,341.000000,1497.000000,1922.000000,155.000000,247.000000,1434.000000,956.000000,2301.000000,252.000000,167.000000
mean,3.315746,3.331504,3.402787,3.653230,3.539559,3.594427,3.617525,3.634012,3.539913,3.662419,3.451345,3.586034,3.695652,3.750000,3.527223,3.569668,3.465798,3.511732,3.469076,3.487767,3.386350,3.573220,3.587424,3.328934,3.621088,3.523322,3.505218,3.506139,3.459046,3.425141,3.492244,3.360069,3.634529,3.591087,3.594756,3.413844,3.522312,3.474820,3.530333,3.521515,3.596774,3.658097,3.506304,3.513889,3.483095,3.390000,3.498756,3.386203,3.554217,2.596290,3.372977,3.434673,3.574163,3.504384,3.445748,3.321977,3.516129,3.754839,3.550607,3.246165,3.304393,3.702303,3.551587,3.688623
std,0.838112,0.798148,0.772411,0.766295,0.774993,0.751541,0.741569,0.742563,0.759350,0.736272,0.820442,0.761377,0.689956,0.693813,0.752776,0.748531,0.781914,0.791587,0.804950,0.787274,0.811147,0.766297,0.763905,0.824530,0.749694,0.804290,0.759981,0.781911,0.749416,0.808894,0.783754,0.828641,0.672791,0.752221,0.739458,0.761064,0.803551,0.777298,0.721219,0.764493,0.689882,0.742555,0.811341,0.765410,0.786909,0.825154,0.759333,0.798124,0.756284,2.595736,0.798909,0.783651,0.758355,0.774139,0.751951,0.837302,0.802780,0.808574,0.713167,0.901307,0.846565,0.626612,0.768735,0.590121
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4

### '영향을 미쳤던' 정보 채널 추출 

In [119]:
temp = mcr_raw_quest_copy.항목설명.apply(lambda x: True if '휴대폰 선택 시, 영향을 미쳤던' in str(x) else False)
temp = temp[temp==1].index
mcr_impact_quest_phone = mcr_raw_quest.iloc[temp]
mcr_impact_quest_phone_id = mcr_impact_quest_phone.항목ID
mcr_impact_quest_phone.head()

,항목ID,항목명,항목분류,항목유형,항목설명
1474,J_02_01_009#1,휴대폰 브랜드 선택시 영향을 미쳤던 정보 채널,제품>통신>휴대폰,복수,"휴대폰 선택 시, 영향을 미쳤던 정보채널을 모두 골라주십시오."
1475,J_02_01_009#2,휴대폰 브랜드 선택시 영향을 미쳤던 정보 채널,제품>통신>휴대폰,복수,"휴대폰 선택 시, 영향을 미쳤던 정보채널을 모두 골라주십시오."
1476,J_02_01_009#3,휴대폰 브랜드 선택시 영향을 미쳤던 정보 채널,제품>통신>휴대폰,복수,"휴대폰 선택 시, 영향을 미쳤던 정보채널을 모두 골라주십시오."
1477,J_02_01_009#4,휴대폰 브랜드 선택시 영향을 미쳤던 정보 채널,제품>통신>휴대폰,복수,"휴대폰 선택 시, 영향을 미쳤던 정보채널을 모두 골라주십시오."
1478,J_02_01_009#5,휴대폰 브랜드 선택시 영향을 미쳤던 정보 채널,제품>통신>휴대폰,복수,"휴대폰 선택 시, 영향을 미쳤던 정보채널을 모두 골라주십시오."


In [121]:
mcr_impact_answer_phone = mcr_raw_answer.loc[:,mcr_impact_quest_phone_id]
mcr_impact_answer_phone.describe() 

,J_02_01_009#1,J_02_01_009#2,J_02_01_009#3,J_02_01_009#4,J_02_01_009#5,J_02_01_009#6,J_02_01_009#7,J_02_01_009#8,J_02_01_009#9,J_02_01_009#10,J_02_01_009#11,J_02_01_009#12,J_02_01_009#13
count,3994.000000,3139.000000,1876.000000,1021.000000,552.000000,350.000000,225.000000,77.000000,24.000000,4.0,0.0,0.0,0.0
mean,4.374061,5.498885,9.431770,10.567091,11.070652,11.305714,11.844444,12.753247,15.416667,12.0,NaN,NaN,NaN
std,12.666059,7.213196,7.275979,8.029398,9.248166,6.677656,5.829250,9.861082,17.594260,0.0,NaN,NaN,NaN
min,1.000000,1.000000,2.000000,4.000000,5.000000,7.000000,9.000000,10.000000,11.000000,12.0,NaN,NaN,NaN
25%,1.000000,2.000000,7.000000,8.000000,9.000000,10.000000,11.000000,11.000000,12.000000,12.0,NaN,NaN,NaN
50%,1.000000,2.000000,9.000000,10.000000,10.000000,11.000000,12.000000,12.000000,12.000000,12.0,NaN,NaN,NaN
75%,1.000000,9.000000,11.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.0,NaN,NaN,NaN
max,99.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,12.0,NaN,NaN,NaN


> 정리

In [62]:
jepum_adver_df.iloc[:,0] # 고객의 휴대폰 단말기 광고 관심도

0       2
1       2
2       5
3       2
4       5
       ..
3995    4
3996    3
3997    5
3998    3
3999    2
Name:  휴대폰 단말기 광고, Length: 4000, dtype: int64

In [63]:
mcr_impact_answer['J_02_01_008'] # 휴대폰 선택시 광고가 제품 선택여부에 주는 영향력

0       1.0
1       4.0
2       4.0
3       3.0
4       3.0
       ... 
3995    4.0
3996    4.0
3997    4.0
3998    4.0
3999    4.0
Name: J_02_01_008, Length: 4000, dtype: float64

In [122]:
mcr_impact_answer_phone.mean() # 휴대폰 선택시 영향을 미친 정보 채널 (효과적이었던 광고매체)

J_02_01_009#1      4.374061
J_02_01_009#2      5.498885
J_02_01_009#3      9.431770
J_02_01_009#4     10.567091
J_02_01_009#5     11.070652
J_02_01_009#6     11.305714
J_02_01_009#7     11.844444
J_02_01_009#8     12.753247
J_02_01_009#9     15.416667
J_02_01_009#10    12.000000
J_02_01_009#11          NaN
J_02_01_009#12          NaN
J_02_01_009#13          NaN
dtype: float64

In [ ]:
for i in mcr_impact_quest_phone_id:
  print(i, mcr_raw_info[mcr_raw_info.항목ID==i].항목코드, mcr_raw_info[mcr_raw_info.항목ID==i].항목코드명)
  mcr_impact_answer_phone
#mcr_raw_info.항목ID==i